# MultiFold Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
### Generic imports
import os
import numpy as np
import pandas as pd
import scipy.stats as stats
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import MaxNLocator

### ML imports
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as preprocessing
import tensorflow as tf

### Custom functions
from omnifold import *
from omnifold.utilities import *

In [3]:
%matplotlib agg
%matplotlib agg
# use this if you want to suppress plots in the Jupyter notebook

In [4]:
! nvidia-smi -L # list GPUs available

GPU 0: Quadro RTX 6000 (UUID: GPU-26dfa6b3-88ef-1e87-ba91-3ca0255166ab)
GPU 1: Quadro RTX 6000 (UUID: GPU-2c315734-0ffc-e8d8-40fb-29a2ccd25ab5)
GPU 2: Quadro RTX 6000 (UUID: GPU-057fcb77-7816-2825-8db8-b78905930b06)
GPU 3: Quadro RTX 6000 (UUID: GPU-dc56feb4-523f-6ded-e577-4416c9dc6561)


In [5]:
### GPU Setup
os.environ["CUDA_VISIBLE_DEVICES"] = "3" # pick a number < 4 on ML4HEP; < 3 on Voltan 
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

2021-11-17 10:07:32.876959: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-17 10:07:32.906951: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:c1:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2021-11-17 10:07:32.907401: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-11-17 10:07:32.909667: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-11-17 10:07:32.911600: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2021-11-17 10:07:32.912470: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2

In [6]:
plot_setup()
plt.rcParams.update({"font.family": "serif",})

In [7]:
plot_dir = './plots/'

### Load files

Since we're not using data yet, "MC" = Powheg + Pythia8 MC16e and "data" = Sherpa 2.2.1 MC16e.

In [8]:
folder = './omnifold_data/'

In [9]:
f_mc = uproot.open(folder+"ZjetOmnifold_Jun25_PowhegPythia_mc16e_slim.root")
f_data = uproot.open(folder+"ZjetOmnifold_Jun25_Sherpa221_mc16e_slim.root")

In [10]:
columns = f_mc['OmniTree'].keys()

These columns are the same for both samples, since they're both MC:

In [11]:
f_mc['OmniTree'].keys() == f_data['OmniTree'].keys()

True

Note that variables ending in `_tracks` require MultiIndex-ing. It's possible to combine these with flat variables, but it does increase the size of the DataFrame considerably, so let's ignore the MultiIndex variables for now. 

In [12]:
mc = f_mc['OmniTree'].arrays(
    [col for col in columns if not col.endswith("_tracks")],
    library="pd")

data = f_data['OmniTree'].arrays(
    [col for col in columns if not col.endswith("_tracks")],
    library="pd")

print("MC has {:,} events with {} columns.".format(mc.shape[0],mc.shape[1]))
print("Data has {:,} events with {} columns.".format(data.shape[0],data.shape[1]))

MC has 416,725 events with 57 columns.
Data has 3,578,623 events with 57 columns.


### Apply event selection

In [13]:
mc['pass190'] = mc['pass190'].astype('bool')
mc['truth_pass190'] = mc['truth_pass190'].astype('bool')
data['pass190'] = data['pass190'].astype('bool')
data['truth_pass190'] = data['truth_pass190'].astype('bool')

The sample we're using already has these cuts applied, but we'll keep them in for future datasets. 

In [14]:
mc = mc[(mc.pass190 | mc.truth_pass190)]
data = data[data.pass190]
data_truth = data[data.truth_pass190]

Normalize the weights:

In [15]:
for weights in [mc.weight_mc, mc.weight, data_truth.weight_mc, data.weight]:
    weights /= np.mean(weights)

### Load IBU histograms to get binning

In [16]:
file_labels = [
    'Ntracks_trackj1', 'Ntracks_trackj2', 'm_trackj1', 'm_trackj2',
    'pT_trackj1', 'pT_trackj2', 'y_trackj1', 'y_trackj2', 'phi_trackj1',
    'phi_trackj2', 'tau1_trackj1', 'tau1_trackj2', 'tau2_trackj1',
    'tau2_trackj2', 'tau3_trackj1', 'tau3_trackj2', 'pT_ll', 'y_ll', 'pT_l1',
    'pT_l2', 'eta_l1', 'eta_l2', 'phi_l1', 'phi_l2'
]
plot_labels = [
    r'Leading track jet $n_{\textrm{ch}}$ ',
    r'Subleading track jet $n_{\textrm{ch}}$', 'Leading track jet $m$ [GeV]',
    r'Subleading track jet $m$ [GeV]', r'Leading track jet $p_T$ [GeV]',
    r'Subleading track jet $p_T$ [GeV]', r'Leading track jet $y$',
    r'Subleading track jet $y$', r'Leading track jet $\phi$',
    r'Subleading track jet $\phi$', r'Leading track jet $\tau_1$',
    r'Subleading track jet $\tau_1$', r'Leading track jet $\tau_2$',
    r'Subleading track jet $\tau_2$', r'Leading track jet $\tau_3$',
    r'Subleading track jet $\tau_3$', r'$p^{\mu \mu}_T$ [GeV]',
    r'$y_{\mu \mu}$', r'$p^{\mu 1}_{T}$ [GeV]', r'$p^{\mu 2}_{T}$ [GeV]',
    '$\eta_{\mu 1}$', '$\eta_{\mu 2}$', '$\phi_{\mu 1}$', '$\phi_{\mu 2}$'
]

IBU_hists = uproot.open(folder+'unfoldingPlotsJune14_UnfoldedHists.root')

bins = []
for label in file_labels:
    bins += [IBU_hists['SherpaUnfoldWPythia_2018_'+label].to_numpy()[1]]
    
labels_and_bins = zip(file_labels, plot_labels, bins)

ibu_hists = []
for file_label, plot_label, plot_bins in labels_and_bins:
    ibu_hists += [{
        'file_label': file_label,
        'plot_label': plot_label,
        'bins': plot_bins
    }]

In [17]:
### Replace bins with Laura's new binning! 
ibu_hists[0]['bins'] = np.array([1, 7, 11, 15, 20, 30, 40]) # leading jet n_charged_tracks
ibu_hists[4]['bins'] = np.array([0,  50,  100,  150,  200,  300,  1000]) # leading jet pT
ibu_hists[10]['bins'] = np.array([0,  0.05,  0.1,  0.17,  0.25,  0.35,  0.5,  0.9]) # leading jet tau_1

### Apply unfolding

In [18]:
K.clear_session()

In [19]:
print("Unfolding {} variables.".format(len(ibu_hists)))

Unfolding 24 variables.


In [20]:
save_label0 = 'example'

In [21]:
mc_truth_plots = [None] * len(ibu_hists)
mc_reco_plots = [None] * len(ibu_hists)
data_truth_plots = [None] * len(ibu_hists)
data_reco_plots = [None] * len(ibu_hists)

mc_truth_hists = [None] * len(ibu_hists)
mc_reco_hists = [None] * len(ibu_hists)
data_truth_hists = [None] * len(ibu_hists)
data_reco_hists = [None] * len(ibu_hists)

# z-score standardization of data
sim_truth_z = [None] * len(ibu_hists)
sim_reco_z = [None] * len(ibu_hists)
data_reco_z = [None] * len(ibu_hists)

Plot distributions before unfolding:

In [22]:
for i in tqdm(range(len(ibu_hists))):
    file_label = ibu_hists[i]['file_label']
    
    dummyval = -99
    
    ### Add in 200 GeV cuts for plotting only 
    mc_pt200 = mc[(mc.truth_pT_ll > 200) | (mc.pT_ll > 200)]
    data_truth_pt200 = data_truth[data_truth.truth_pT_ll > 200]
    data_pt200 = data[data.pT_ll > 200]
    
    mc_truth_plots[i] = np.where(mc_pt200.truth_pass190, mc_pt200['truth_'+file_label], dummyval)
    mc_reco_plots[i] = np.where(mc_pt200.pass190, mc_pt200[file_label], dummyval)
    data_truth_plots[i] = data_truth_pt200['truth_'+file_label]
    data_reco_plots[i] = data_pt200[file_label]
    
    save_label0 = 'weight_truncation'
    bins = ibu_hists[i]['bins']
    x_label = ibu_hists[i]['plot_label']
    file_label = ibu_hists[i]['file_label']
    os.makedirs(plot_dir+'MultiFold/'+file_label, exist_ok=True)
    save_label = plot_dir+'MultiFold/'+file_label+'/'+save_label0

    plot_distributions(
        sim_truth=mc_truth_plots[i],
        sim_reco=mc_reco_plots[i],
        sim_truth_weights_MC=mc_pt200.weight_mc,
        sim_reco_weights_MC=mc_pt200.weight,
        data_truth=data_truth_plots[i],
        data_reco=data_reco_plots[i],
        data_truth_weights_MC=data_truth_pt200.weight_mc,
        data_reco_weights_MC=data_pt200.weight,
        bins=bins,
        x_label=x_label,
        save_label=save_label
    )
    
    mc_truth_hists[i] = np.where(mc.truth_pass190, mc['truth_'+file_label], dummyval)
    mc_reco_hists[i] = np.where(mc.pass190, mc[file_label], dummyval)
    data_truth_hists[i] = data_truth['truth_'+file_label]
    data_reco_hists[i] = data[file_label]
    
    sim_truth_z[i], sim_reco_z[i], data_reco_z[i] = standardize(
        np.array(mc_truth_hists[i]), 
        np.array(mc_reco_hists[i]), 
        np.array(data_reco_hists[i]))

 83%|████████▎ | 20/24 [01:57<00:23,  5.82s/it]/clusterfs/ml4hep/mpettee/ATLASOmniFold/omnifold/plot.py:51: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(nrows=2,
100%|██████████| 24/24 [02:15<00:00,  5.65s/it]


Unfold:

In [23]:
import time
t0 = time.time()

weights, _ = multifold(iterations=5,
                       sim_truth=sim_truth_z,
                       sim_reco=sim_reco_z,
                       sim_truth_weights_MC=mc.weight_mc,
                       sim_reco_weights_MC=mc.weight,
                       data_reco=data_reco_z,
                       data_reco_weights_MC=data.weight,
                       dummyval=dummyval,
                       verbose=0)

print("Finished in {:.2f} seconds.".format(time.time() - t0))

2021-11-17 10:10:08.521979: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2021-11-17 10:10:08.531152: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2994280000 Hz
2021-11-17 10:10:08.532205: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x558019275a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-17 10:10:08.532232: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-11-17 10:10:08.669663: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x557fd4a54430 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2021-11-17 10:10:08.669712: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 6000, Compute Capability 7.5
2021-11-17 10:10:08.671363: I 

Finished in 124.18689942359924 seconds.


Plot results from unfolding:

In [25]:
filter = (mc.truth_pT_ll > 200) | (mc.pT_ll > 200)

for i in tqdm(range(len(ibu_hists))):
    # print(ibu_hists[i]['file_label'] + " Results\n")
    bins = ibu_hists[i]['bins']
    x_label = ibu_hists[i]['plot_label']
    file_label = ibu_hists[i]['file_label']
    save_label = plot_dir + '/MultiFold/' + file_label + '/' + save_label0 + '-MultiFold-' + file_label

    plot_results(sim_truth=mc_truth_plots[i],
                 sim_reco=mc_reco_plots[i],
                 sim_truth_weights_MC=mc_pt200.weight_mc,
                 sim_reco_weights_MC=mc_pt200.weight,
                 data_truth=data_truth_plots[i],
                 data_reco=data_reco_plots[i],
                 data_truth_weights_MC=data_truth_pt200.weight_mc,
                 data_reco_weights_MC=data_pt200.weight,
                 weights=weights[:,:,filter],
                 bins=bins,
                 x_label=x_label,
                 flavor_label='MultiFold',
                 save_label=save_label)

100%|██████████| 24/24 [07:18<00:00, 18.28s/it]
